### Test with ELO threshold

In [28]:
# from model.game import Game

# file = open("../data/data.pgn", mode="r")
# white_elo, black_elo, game = 0, 0, ""
# elo_thres = 2000

# games = []
# for i, row in enumerate(file):

#     if "WhiteElo" in row:  # white elo
#         try:
#             white_elo = int(row.split('"')[1])
#         except:
#             #print(i, white_elo)
#             white_elo = 0

#     elif "BlackElo" in row:  # black elo
#         try:
#             black_elo = int(row.split('"')[1])
#         except:
#             #print(i, black_elo)
#             black_elo = 0

#     elif row[0] == '1':  # game: save and reset
#         if white_elo > elo_thres and black_elo > elo_thres:
#             games.append(Game(white_elo=white_elo, black_elo=black_elo, game=row))
#         white_elo, black_elo, game = 0, 0, ""
        
# print(len(games))

4227


In [29]:
print(games[20])

white_elo: 2041
black_elo: 2041
1. e4 c5 2. Nf3 d6 3. c3 Nf6 4. Bd3 Nc6 5. Bc2 e5 6. h3 Be6 7. O-O Be7 8. Qe2 O-O 9. Rd1 Re8 10. d4 Bc8 11. a4 cxd4 12. cxd4 exd4 13. Nxd4 Nxd4 14. Rxd4 Be6 15. Nc3 Rc8 16. Bf4 Qb6 17. Rad1 Qxb2 18. Nb5 Rxc2 0-1



### Test filter with eval

In [1]:
file = open("../data/data.pgn", mode="r")

games = []
for i, row in enumerate(file):
    ## if not a game, continue
    if row[0] != '1':
        continue
    #if the row doesnt contains evaluation, continue
    if "eval" not in row:
        continue
    games.append(row)

print(len(games))

1717


In [31]:
print(games[5])

1. e4 { [%eval 0.28] } 1... c6 { [%eval 0.54] } 2. Bc4?! { [%eval -0.32] } 2... d5 { [%eval -0.27] } 3. exd5 { [%eval -0.32] } 3... cxd5 { [%eval -0.32] } 4. Bb5+ { [%eval -0.36] } 4... Nc6 { [%eval -0.16] } 5. Bxc6+ { [%eval -0.42] } 5... bxc6 { [%eval -0.6] } 6. Nf3 { [%eval -0.48] } 6... Nf6 { [%eval -0.44] } 7. O-O { [%eval -0.5] } 7... Bg4 { [%eval -0.35] } 8. h3 { [%eval -0.37] } 8... Bh5 { [%eval -0.3] } 9. d3 { [%eval -0.3] } 9... e6 { [%eval -0.31] } 10. Nbd2 { [%eval -0.58] } 10... Bd6 { [%eval -0.53] } 11. Qe1 { [%eval -0.57] } 11... O-O { [%eval -0.32] } 12. a3?! { [%eval -0.94] } 12... Rb8 { [%eval -0.84] } 13. Ne5 { [%eval -0.63] } 13... Qc7 { [%eval -0.74] } 14. d4?! { [%eval -1.46] } 14... c5 { [%eval -1.44] } 15. c3 { [%eval -1.56] } 15... cxd4 { [%eval -1.44] } 16. cxd4 { [%eval -1.45] } 16... Nd7 { [%eval -1.33] } 17. Nxd7 { [%eval -1.21] } 17... Qxd7 { [%eval -1.23] } 18. Nf3?! { [%eval -2.11] } 18... Bxf3 { [%eval -2.0] } 19. gxf3 { [%eval -1.98] } 19... Bc7 { [%ev

In [47]:
# convert each position of a game to a dict with positions and evaluations
import json

def process_move(move_data: str):
    # convert move of the form '1. e4 { [%eval 0.28] ' and return move and eval
    move_data = move_data.split(" { ")
    move = move_data[0].split(". ")[1].replace('?','').replace('!','')
    eval = move_data[1].split("%eval ")[1].split("]")[0].replace('#','')
    return move, eval


def convert_game_to_json(pgn_game: str) -> list[dict]:
    # replace ... with _ for black moves to be able to split
    splits = pgn_game.split("}")
    positions = []
    for move_data in splits:
        if '\n' in move_data:
            break
        move, eval = process_move(move_data)
        if (len(positions)>0):
            move = positions[-1]["moves"] + "_" + move
        positions.append({"moves": move, "eval": eval})
    return positions

convert_game_to_json(games[5])

def convert_games_to_json(games: list[str]) -> list[dict]:
    # convert each game to json
    json_games = []
    for game in games:
        json_games+=convert_game_to_json(game)
    with open('../data/positions.json', 'w') as outfile:
        json.dump(json_games, outfile)
    


In [50]:
convert_games_to_json(games)

### Tests with python chess library

In [55]:
import chess

board = chess.Board()

print(board.fen())

rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1


### Filter games with evaluation and dump to file

In [10]:
file_path = '../data/data.pgn'
save_path = '../data/eval_data.pgn'
eval_file = ''
eval_games = []
with open(file_path, 'r') as f:
    game_rows = []
    for line in f:
        game_rows.append(line)
        if line[0] == '1':
            if 'eval' in line:
                if game_rows[0] == '\n':
                    game_rows.pop(0)
                if game_rows[-1] != '\n':
                    game_rows.append('\n')
                eval_file += ''.join(game_rows)
                eval_games.append(game_rows)
            game_rows = []

with open(save_path, 'w') as f:
    f.write(eval_file)

In [27]:
print(len(eval_games))

1717


In [39]:
import chess.pgn 
import json
import csv

def process_game(game: chess.pgn.Game):
    positions = []
    while not game.is_end():
        game = game.variation(0)
        fen = game.board().fen()
        if game.eval() is not None:
            if game.eval().is_mate():
                eval = '#'+str(game.eval().relative.mate())
            else:
                eval = game.eval().relative.score()
            positions.append({"fen": fen, "eval": eval})
    return positions


def dump_positions_csv(positions: list[dict]):
    with open('../data/positions.csv', 'w', newline='') as csvfile:
        fieldnames = ['fen', 'eval']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for position in positions:
            writer.writerow(position)
    
all_positions = []
with open("../data/eval_data.pgn") as pgn:
    for i in range(1717):
        game = chess.pgn.read_game(pgn)
        if game is None:
            break
        positions = process_game(game)
        all_positions += positions
    dump_positions_csv(all_positions)
      

### Remove positions duplicates

In [2]:
from more_itertools import unique_everseen
with open('../data/positions.csv', 'r') as f, open('../data/positions_unique.csv', 'w') as out_file:
    out_file.writelines(unique_everseen(f))
            


In [5]:
import os

def get_all_files_in_dir(dir_path: str) -> list[str]:
    files = []
    for r, d, f in os.walk(dir_path):
        for file in f:
            if '.pgn' in file:
                files.append(os.path.join(r, file))
    return files

print(get_all_files_in_dir('../data/raw_pgn'))

['../data/raw_pgn\\data.pgn', '../data/raw_pgn\\lichess_Timdov_2023-07-23.pgn']


## Decompress ZTS to PGN

In [7]:
import zstandard as zstd
dctx = zstd.ZstdDecompressor()
submission_path_read = '../data/zst/data_2014_07.zst'
submission_path_save = '../data/raw_pgn/data_2014_07.pgn'
with open(submission_path_read, 'rb') as ifh, open(submission_path_save, 'wb') as ofh:
    dctx.copy_stream(ifh, ofh, write_size=65536)

In [1]:
def get_number_of_games_in_file(file_path: str) -> int:
    nb_games = 0
    with open(file_path) as pgn:
        while True:
            row = pgn.readline()
            if row == '':
                break
            if row[0] == '1':
                nb_games += 1
    return nb_games

print(get_number_of_games_in_file('../data/filtered_pgn/data_2014_07.pgn'))        


175033


In [7]:
import pandas as pd
df = pd.read_csv('../data/csv_positions/data_2013_09.csv')
df.columns[0] == 'fen'

False

In [24]:
import chess.pgn
with open ('../data/filtered_pgn/data_2013_09.pgn') as pgn:
    game = chess.pgn.read_game(pgn)

while game.eval() is None or not game.eval().is_mate():
    game = game.variation(0)
print(game.board())
print(game.eval().relative.score(mate_score=10000))

. . . . . . k .
. . R . . p p .
p . . . . . . .
. p . . . . . p
. . . . . . . .
P P . . . P . .
. . . r . . r P
. . . . . K . .
9993


In [29]:
pieces = ['r', 'n', 'b', 'q', 'k', 'p', 'R', 'N', 'B', 'Q', 'K', 'P']
import tensorflow as tf
import numpy as np
def convert_fen_row_to_array(fen_row: str, piece: list[str]):
        array = np.zeros(8, dtype=np.uint8)
        idx = 0
        for char in fen_row:
            if char.isdigit():
                idx += int(char)
            elif char == piece:
                array[idx] = 1
                idx += 1
            else:
                idx += 1
        return array

def convert_fen_to_tensor(fen: str):
    # environ 0.3 à 0.4ms par fen
    game = fen.split(' ')[0]
    rows = game.split('/')
    tensor_array = np.zeros((12, 8, 8), dtype=np.uint8)
    for piece_idx, piece in enumerate(pieces):
        for row_idx, row in enumerate(rows):
            tensor_array[piece_idx, row_idx, :] = convert_fen_row_to_array(row, piece)
    # rehape to 8,8,12
    tensor_array = np.moveaxis(tensor_array, 0, -1)
    return tf.convert_to_tensor(tensor_array)

fen_tensor = convert_fen_to_tensor('rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1')
fen_tensor.shape

TensorShape([8, 8, 12])

In [ ]:
import time
from utils.converter import convert_fen_to_array
time_start = time.time()
for i in range(1000):
    fen_tensor = convert_fen_to_array('rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1')
time_end = time.time()
print(time_end - time_start)
print(fen_tensor.shape)